<a href="https://colab.research.google.com/github/Atik07/practical-machine-learning/blob/main/tansfer_learning_svm_fmnist_w_mlp_over_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras

import sklearn
from sklearn import svm #SVM
from sklearn import metrics

%matplotlib inline

In [ ]:
#importing mnist
(mnist_x_train, mnist_y_train), (mnist_x_test, mnist_y_test) = tf.keras.datasets.mnist.load_data()

In [ ]:
#importing fashion-mnist
(fmnist_x_train, fmnist_y_train), (fmnist_x_test, fmnist_y_test) = tf.keras.datasets.fashion_mnist.load_data()

In [ ]:
size_train_set = 6000
size_test_set = 300

###SVM

In [ ]:
class p_svm:
  svm_models = [svm.SVC(kernel='linear', C=1.0), svm.SVC(kernel='rbf', gamma=0.7, C=1.0)]
  
  def train_svm(self,x_train,y_train):
    for model in self.svm_models:
      model.fit(x_train, y_train)

  def analyse_svm(self,x_test,y_test):
    for model in self.svm_models:
      y_pred = (model.predict(x_test))
      accuracy = metrics.accuracy_score(y_test, y_pred)
      print('For ',model.kernel, 'kernel -- accuracy: ',accuracy)

###SVM over f-MNIST

In [ ]:
(x_train, y_train, x_test, y_test) = (fmnist_x_train[:size_train_set], fmnist_y_train[:size_train_set], fmnist_x_test[:size_test_set], fmnist_y_test[:size_test_set])

x_train=x_train.reshape(x_train.shape[0],x_train.shape[1]*x_train.shape[2])
x_test=x_test.reshape(x_test.shape[0],x_test.shape[1]*x_test.shape[2])

x_train=x_train/255.0
x_test=x_test/255.0

perform_svm = p_svm()
perform_svm.train_svm(x_train, y_train)
perform_svm.analyse_svm(x_test, y_test)

For  linear kernel -- accuracy:  0.86
For  rbf kernel -- accuracy:  0.19666666666666666


###MLP over MNIST

In [ ]:
(x_train, y_train, x_test, y_test) = (mnist_x_train[:size_train_set], mnist_y_train[:size_train_set], mnist_x_test[:size_test_set], mnist_y_test[:size_test_set])

x_train = x_train/255.0
x_test = x_test/255.0

In [ ]:
(img_height, img_width) = (x_train.shape[1],x_train.shape[2])
#Set up the layers
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(img_height, img_width), name='flatten-layer'), # to Flatten 28*28 to 784
    tf.keras.layers.Dense(img_height*img_width/4, activation='relu' , name='hidden-1'),
    tf.keras.layers.Dense(img_height*img_width/8, activation='sigmoid' , name='hidden-2'),
    tf.keras.layers.Dense(img_height*img_width/16, activation='sigmoid' , name='hidden-3'),
    tf.keras.layers.Dense(32, activation='relu', name='hidden-4'),
    tf.keras.layers.Dense(10)
])

In [ ]:
#Compile the model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten-layer (Flatten)      (None, 784)               0         
_________________________________________________________________
hidden-1 (Dense)             (None, 196)               153860    
_________________________________________________________________
hidden-2 (Dense)             (None, 98)                19306     
_________________________________________________________________
hidden-3 (Dense)             (None, 49)                4851      
_________________________________________________________________
hidden-4 (Dense)             (None, 32)                1600      
_________________________________________________________________
dense (Dense)                (None, 10)                330       
Total params: 179,947
Trainable params: 179,947
Non-trainable params: 0
__________________________________________________

In [ ]:
#Train the model
model.fit(x_train, y_train, epochs=10)


Epoch 1/10
188/188 [==============================] - 4s 2ms/step - loss: 1.3664 - accuracy: 0.6098
Epoch 2/10
188/188 [==============================] - 0s 2ms/step - loss: 0.4105 - accuracy: 0.9012
Epoch 3/10
188/188 [==============================] - 0s 2ms/step - loss: 0.2243 - accuracy: 0.9427
Epoch 4/10
188/188 [==============================] - 0s 2ms/step - loss: 0.1521 - accuracy: 0.9630
Epoch 5/10
188/188 [==============================] - 0s 2ms/step - loss: 0.1089 - accuracy: 0.9715
Epoch 6/10
188/188 [==============================] - 0s 2ms/step - loss: 0.0785 - accuracy: 0.9822
Epoch 7/10
188/188 [==============================] - 0s 2ms/step - loss: 0.0539 - accuracy: 0.9885
Epoch 8/10
188/188 [==============================] - 0s 2ms/step - loss: 0.0465 - accuracy: 0.9893
Epoch 9/10
188/188 [==============================] - 0s 2ms/step - loss: 0.0278 - accuracy: 0.9947
Epoch 10/10
188/188 [==============================] - 0s 2ms/step - loss: 0.0213 - accuracy: 0.9958

In [ ]:
#Evaluate accuracy
test_loss, test_acc = model.evaluate(x_test,  y_test)

print('\nTest accuracy:',test_acc, '\nTest loss:',test_loss)

10/10 [==============================] - 0s 2ms/step - loss: 0.1466 - accuracy: 0.9600

Test accuracy: 0.9599999785423279 
Test loss: 0.14661118388175964


###transforming fashion-MNIST
pop output layer of trained MLP

f-MNIST through the model

In [ ]:
# remove the last layer of the model
model.pop() 

In [ ]:
# feature extraction from the above model

feature_extractor = keras.Model(
    inputs=model.inputs,
    outputs=model.get_layer(name="hidden-4").output,
)

(t_fmnist_x_train, t_fmnist_y_train) = (feature_extractor(fmnist_x_train[:size_train_set]) , fmnist_y_train[:size_train_set] )
(t_fmnist_x_test, t_fmnist_y_test) = (feature_extractor(fmnist_x_test[:size_test_set]), fmnist_y_test[:size_test_set])

###SVM over transformed f-MNIST 

In [ ]:
(x_train, y_train, x_test, y_test) = (t_fmnist_x_train, t_fmnist_y_train, t_fmnist_x_test, t_fmnist_y_test)

perform_svm = p_svm()
perform_svm.train_svm(x_train, y_train)
perform_svm.analyse_svm(x_test, y_test)